## Start

Let's first load the dataset (which can be found [here](https://www.kaggle.com/stackoverflow/stacksample)) into pandas.

We'll start by grabbing a list of the question titles.

In [1]:
import pandas as pd

df = (pd.read_csv("archive/Questions.csv", nrows=1_000_000,
                  encoding="ISO-8859-1", usecols=['Title', 'Id']))
titles = [_ for _ in df['Title']]

In [2]:
def has_golang(text):
    return " go " in text

g = (title for title in titles if has_golang(title))
[next(g) for i in range(2)]

['Where does Console.WriteLine go in ASP.NET?',
 'Should try...catch go inside or outside a loop?']

It doesn't work too well. So let us try spacy first. You might need to install the language model first.

```
python -m spacy download en_core_web_sm
```

In [2]:
import spacy 

nlp = spacy.load("en_core_web_sm")

Let's first see how it works.

In [3]:
[t for t in nlp("My name is Vincent.")]

[My, name, is, Vincent, .]

In [4]:
doc = nlp("My name is Vincent.")

In [5]:
t = doc[0]

In [6]:
from spacy import displacy

displacy.render(doc)

In [7]:
for t in nlp("Where does Console.WriteLine go in ASP.NET?"):
    print(t, t.pos_, t.dep_)

Where SCONJ advmod
does VERB ROOT
Console PROPN nsubj
. PUNCT punct
WriteLine PROPN nsubj
go VERB ROOT
in ADP prep
ASP.NET PROPN pobj
? PUNCT punct


# Back to Detecting Golang

Let's now use it one more time for our problem of detecting the `go` language. I'll also go ahead and write the code in a more performant way.

In [8]:
nlp = spacy.load("en_core_web_sm")

In [10]:
df = (pd.read_csv("archive/Questions.csv", nrows=2_000_000,
                  encoding="ISO-8859-1", usecols=['Title', 'Id']))

titles = [_ for _ in df.loc[lambda d: d['Title'].str.lower().str.contains("go")]['Title']]

In [11]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [12]:
%%time

def has_golang(doc):
    for t in doc:
        if t.lower_ in ["go", "golang"]:
            if t.pos_ == "NOUN":
                return True 
    return False

g = (doc for doc in nlp.pipe(titles) if has_golang(doc))
[next(g) for i in range(30)]

CPU times: user 10.1 s, sys: 11.5 ms, total: 10.2 s
Wall time: 10.2 s


[Can I append an Ajax requestXML object to my document tree all in one go?,
 How do I disable multiple listboxes in one go using jQuery?,
 Trouble reading from a socket in go,
 Is there a way to find a specific file and then change into the directory containing it in one go?,
 How many records can be loaded into Salesforce using Apex Data Loader in one go?,
 How can I run multiple inserts with NHibernate in one go?,
 what's the state of go language IDE support?,
 Decrypt many PDFs in one go using pdftk,
 Generating Random Numbers in Go,
 making generic algorithms in go,
 How do I allocate memory for an array in the go programming language?,
 Pass variables into Thread on the go?,
 If two options are the same go to next option in second select,
 How to clone an element and insert it multiple times in one go?,
 How to use LevelDB in go?,
 Auto-complete with go-mode,
 Why is this Go code blocking?,
 How do I use an custom int type with range in go?,
 importing packages in go,
 Can Terraco

This works! Now let's write some pandas code that will help us with our benchmarks.

![](images/these-2.png)

In [15]:
df_tags = pd.read_csv("archive/Tags.csv")
go_ids = df_tags.loc[lambda d: d['Tag'] == 'go']['Id']

def has_go_token(doc):
    for t in doc:
        if t.lower_ in ['go', 'golang']:
            if t.pos_ != 'VERB':
                return True
    return False

all_go_sentences = df.loc[lambda d: d['Id'].isin(go_ids)]['Title'].tolist()
detectable = [d.text for d in nlp.pipe(all_go_sentences) if has_go_token(d)]

non_detectable = (df
                  .loc[lambda d: ~d['Id'].isin(go_ids)]
                  .loc[lambda d: d['Title'].str.lower().str.contains("go")]
                  ['Title']
                  .tolist())

non_detectable = [d.text for d in nlp.pipe(non_detectable) if has_go_token(d)]

len(all_go_sentences), len(detectable), len(non_detectable)

(1858, 918, 82)

Nice, we get some numbers that can result in a meaningful benchmark.

We can calculate precision/recall like stats by running the code below. You can put a forloop around it if you want but as it is you can fiddle around with the `has_go_token` function to see how well it performs.

In [16]:
model_name = "en_core_web_sm"
model = spacy.load(model_name, disable=["ner"])

def has_go_token(doc):
    for t in doc:
        if t.lower_ in ["go", "golang"]:
            if t.pos_ != "VERB":
                return True
    return False

method = "not-verb-but-pobj"

correct = sum(has_go_token(doc) for doc in model.pipe(detectable))
wrong = sum(has_go_token(doc) for doc in model.pipe(non_detectable))
precision = correct/(correct + wrong)
recall = correct/len(detectable)
accuracy = (correct + len(non_detectable) - wrong)/(len(detectable) + len(non_detectable))

f"{precision},{recall},{accuracy},{model_name},{method}" # this is logged

'0.918,1.0,0.918,en_core_web_sm,not-verb-but-pobj'

Enjoy playing around with this.